In [1]:
# importation des bibliothèques
import pandas as pd
import numpy as np

In [2]:
# load datasets
ticket_data = pd.read_csv('/content/drive/MyDrive/ticket_data.csv')
cities = pd.read_csv('/content/drive/MyDrive/cities.csv')
providers = pd.read_csv('/content/drive/MyDrive/providers.csv')
stations = pd.read_csv('/content/drive/MyDrive/stations.csv')

In [3]:
ticket_data['arrival_ts'] = pd.to_datetime(ticket_data['arrival_ts'])
ticket_data['departure_ts'] = pd.to_datetime(ticket_data['departure_ts'])

**On calcule ici la durée des voyages**

In [14]:
# ajouter la période du trajets
ticket_data['durée'] = ticket_data['arrival_ts'] - ticket_data['departure_ts'] 

In [15]:
ticket_data['durée'] = pd.to_timedelta(ticket_data['durée']).astype('timedelta64[m]').astype(int)

In [16]:
ticket_data

,id,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city,durée
0,6795025,8385,NaN,NaN,2017-10-13 14:00:00+00:00,2017-10-13 20:10:00+00:00,4550,2017-10-01 00:13:31.327+00,NaN,NaN,611,542,370
1,6795026,9,63.0,1044.0,2017-10-13 13:05:00+00:00,2017-10-14 06:55:00+00:00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},611,542,1070
2,6795027,8377,5905.0,6495.0,2017-10-13 13:27:00+00:00,2017-10-14 21:24:00+00:00,7400,2017-10-01 00:13:40.212+00,"{798,798,6794,6246}","{8377,8376}",611,542,1917
3,6795028,8377,5905.0,6495.0,2017-10-13 13:27:00+00:00,2017-10-14 11:02:00+00:00,13500,2017-10-01 00:13:40.213+00,"{798,798,6794,6246}","{8377,8376}",611,542,1295
4,6795029,8381,5905.0,6495.0,2017-10-13 21:46:00+00:00,2017-10-14 19:32:00+00:00,7710,2017-10-01 00:13:40.213+00,"{5983,5983}",{8380},611,542,1306
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74163,6869182,13,279.0,10729.0,2017-10-27 02:30:00+00:00,2017-10-27 16:00:00+00:00,2590,2017-10-19 10:35:42.943+00,"{408,408}",{8371},628,562,810
74164,6869185,9,279.0,304.0,2017-10-27 07:00:00+00:00,2017-10-27 13:30:00+00:00,2700,2017-10-19 10:35:42.943+00,"{1105,1105}",{9},628,562,390
74165,6869187,8371,10642.0,304.0,2017-10-27 08:30:00+00:00,2017-10-27 15:15:00+00:00,3000,2017-10-19 10:35:42.943+00,"{1105,1105}",{9},628,562,405
74166,6869189,13,279.0,304.0,2017-10-27 13:25:00+00:00,2017-10-27 21:10:00+00:00,2880,2017-10-19 10:35:42.943+00,"{863,863}",{13},628,562,465


## **Extraction des données** 


***Les prix de tickets par trajet :***

In [17]:
ticket_data.groupby(['o_city','d_city']).agg({'price_in_cents':['mean','max','min']})

price_in_cents              
                        mean    max    min
o_city d_city                             
5      23       20320.000000  22000  18600
6      227      11755.000000  13650   9860
       504       4042.666667   8920   2000
       628       2797.500000   3190   2600
       845        864.626866   2420    700
...                      ...    ...    ...
11938  126       4204.861111   5350   3000
12124  1064      6950.000000   7900   6000
12166  857       5300.000000   5300   5300
12190  639        688.888889    850    600
       8937      9800.000000   9800   9800

[1437 rows x 3 columns]

***Les durée de voyages par minutes par trajet :*** 

In [18]:
ticket_data.groupby(['o_city','d_city']).agg({'durée':['mean','max','min']})

durée            
                      mean   max   min
o_city d_city                         
5      23       618.800000   954   533
6      227      822.500000   901   744
       504      497.400000   740   336
       628      730.000000   870   580
       845       79.910448   251    60
...                    ...   ...   ...
11938  126      445.277778  2120   330
12124  1064    1030.000000  1390   670
12166  857     1315.000000  1315  1315
12190  639       88.888889   160    70
       8937     550.000000   550   550

[1437 rows x 3 columns]

### **j'ai fait les calculs de longuers trajets de différents voyages**

In [32]:
import geopy.distance
ticket_data['distance'] = 0
for i in ticket_data.index:
  coords_1 = (cities.loc[cities['id']==ticket_data.loc[i,'o_city'],'latitude'].values[0], cities.loc[cities['id']==ticket_data.loc[i,'o_city'],'longitude'].values[0])
  coords_2 = (cities.loc[cities['id']== ticket_data.loc[i,'d_city'],'latitude'].values[0], cities.loc[cities['id']==ticket_data.loc[i,'d_city'],'longitude'].values[0])
  ticket_data.loc[i,'distance'] = geopy.distance.vincenty(coords_1, coords_2).km

In [33]:
ticket_data

,id,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city,durée,distance
0,6795025,8385,NaN,NaN,2017-10-13 14:00:00+00:00,2017-10-13 20:10:00+00:00,4550,2017-10-01 00:13:31.327+00,NaN,NaN,611,542,370,503.136831
1,6795026,9,63.0,1044.0,2017-10-13 13:05:00+00:00,2017-10-14 06:55:00+00:00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},611,542,1070,503.136831
2,6795027,8377,5905.0,6495.0,2017-10-13 13:27:00+00:00,2017-10-14 21:24:00+00:00,7400,2017-10-01 00:13:40.212+00,"{798,798,6794,6246}","{8377,8376}",611,542,1917,503.136831
3,6795028,8377,5905.0,6495.0,2017-10-13 13:27:00+00:00,2017-10-14 11:02:00+00:00,13500,2017-10-01 00:13:40.213+00,"{798,798,6794,6246}","{8377,8376}",611,542,1295,503.136831
4,6795029,8381,5905.0,6495.0,2017-10-13 21:46:00+00:00,2017-10-14 19:32:00+00:00,7710,2017-10-01 00:13:40.213+00,"{5983,5983}",{8380},611,542,1306,503.136831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74163,6869182,13,279.0,10729.0,2017-10-27 02:30:00+00:00,2017-10-27 16:00:00+00:00,2590,2017-10-19 10:35:42.943+00,"{408,408}",{8371},628,562,810,339.046766
74164,6869185,9,279.0,304.0,2017-10-27 07:00:00+00:00,2017-10-27 13:30:00+00:00,2700,2017-10-19 10:35:42.943+00,"{1105,1105}",{9},628,562,390,339.046766
74165,6869187,8371,10642.0,304.0,2017-10-27 08:30:00+00:00,2017-10-27 15:15:00+00:00,3000,2017-10-19 10:35:42.943+00,"{1105,1105}",{9},628,562,405,339.046766
74166,6869189,13,279.0,304.0,2017-10-27 13:25:00+00:00,2017-10-27 21:10:00+00:00,2880,2017-10-19 10:35:42.943+00,"{863,863}",{13},628,562,465,339.046766


### **j'ai fait la merge entre ticket_data et providers pour avoir le type de transport pour chaque voyage**

In [34]:
df_merge_col = pd.merge(ticket_data, providers,left_on='company', right_on='id', how='inner')

In [40]:
df_merge_col

,id_x,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city,durée,distance,id_y,company_id,provider_id,name,fullname,has_wifi,has_plug,has_adjustable_seats,has_bicycle,transport_type,DistanceRange
0,6795025,8385,NaN,NaN,2017-10-13 14:00:00+00:00,2017-10-13 20:10:00+00:00,4550,2017-10-01 00:13:31.327+00,NaN,NaN,611,542,370,503.136831,8385,11,NaN,bbc,Blablacar,False,False,False,False,carpooling,201-800km
1,6795030,8385,NaN,NaN,2017-10-06 05:30:00+00:00,2017-10-06 08:30:00+00:00,1800,2017-10-01 01:03:18.948+00,NaN,NaN,628,453,180,205.933671,8385,11,NaN,bbc,Blablacar,False,False,False,False,carpooling,201-800km
2,6795031,8385,NaN,NaN,2017-10-06 07:00:42+00:00,2017-10-06 09:30:42+00:00,2150,2017-10-01 01:03:18.948+00,NaN,NaN,628,453,150,205.933671,8385,11,NaN,bbc,Blablacar,False,False,False,False,carpooling,201-800km
3,6795032,8385,NaN,NaN,2017-10-06 07:10:00+00:00,2017-10-06 09:40:00+00:00,1700,2017-10-01 01:03:18.948+00,NaN,NaN,628,453,150,205.933671,8385,11,NaN,bbc,Blablacar,False,False,False,False,carpooling,201-800km
4,6795033,8385,NaN,NaN,2017-10-06 10:00:00+00:00,2017-10-06 12:50:00+00:00,1700,2017-10-01 01:03:18.948+00,NaN,NaN,628,453,170,205.933671,8385,11,NaN,bbc,Blablacar,False,False,False,False,carpooling,201-800km
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74163,6848395,12,283.0,1595.0,2017-10-27 06:00:00+00:00,2017-10-27 17:45:00+00:00,4687,2017-10-14 19:01:07.915+00,"{561,8813}",{71},1061,1477,705,530.176067,12,4,NaN,studentAgency,Student Agency,False,False,False,False,bus,201-800km
74164,6848396,12,283.0,1593.0,2017-10-27 06:00:00+00:00,2017-10-27 17:20:00+00:00,4777,2017-10-14 19:01:07.915+00,"{561,561}",{71},1061,1477,680,530.176067,12,4,NaN,studentAgency,Student Agency,False,False,False,False,bus,201-800km
74165,6852979,12,523.0,86.0,2017-10-30 21:30:00+00:00,2017-10-31 11:45:00+00:00,1990,2017-10-16 08:10:00.399+00,"{458,458}",{13},809,89,855,123.268672,12,4,NaN,studentAgency,Student Agency,False,False,False,False,bus,0-200km
74166,6853589,12,283.0,523.0,2017-12-22 18:30:00+00:00,2017-12-23 04:25:00+00:00,4590,2017-10-16 10:12:35.105+00,"{330,330}",{13},1061,809,595,352.268303,12,4,NaN,studentAgency,Student Agency,False,False,False,False,bus,201-800km


***on fait ici la catégorisation par distance :***

In [39]:
bins = [0, 200, 800, 2000]
names = ['0-200km', '201-800km', '800-2000km', ' 2000+km']
d = dict(enumerate(names, 1))

df_merge_col['DistanceRange'] = np.vectorize(d.get)(np.digitize(df_merge_col['distance'], bins))

**la différence de durée moyen selon le train, le bus et le covoit et selon la distance du trajet**


In [44]:
df_merge_col.groupby(['transport_type','DistanceRange']).agg({'durée':['mean']})

durée
                                     mean
transport_type DistanceRange             
bus            0-200km         613.950886
               201-800km       904.487822
               800-2000km     1652.167121
carpooling     0-200km         117.122374
               201-800km       285.787874
               800-2000km      786.013986
train          0-200km         265.164002
               201-800km       460.659059
               800-2000km      729.517028

**la différence de prix moyen selon le train, le bus et le covoit et selon la distance du trajet**

In [45]:
df_merge_col.groupby(['transport_type','DistanceRange']).agg({'price_in_cents':['mean']})

price_in_cents
                                       mean
transport_type DistanceRange               
bus            0-200km          2182.830918
               201-800km        3494.850022
               800-2000km       6949.950045
carpooling     0-200km          1177.352385
               201-800km        3218.064310
               800-2000km       8626.545455
train          0-200km          3487.900779
               201-800km        9124.779489
               800-2000km      15463.746130